In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
    'minScale': 1, # Changing creates problems
    'width': 1024,
    'height': 900,
});


%matplotlib inline
import seaborn as sns
import numpy as np
import time
from time import sleep
from IPython.display import display, clear_output

sns.set(rc={'figure.figsize': (15, 6)})
np.set_printoptions(precision=5, threshold=100)

# Set up some variables in case we forget to define them later
import epics

temperature_pv = epics.PV('PYEPICS_DEMO:TEMPERATURE')
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')
wave_pv = epics.PV('PYEPICS_DEMO:WAVE')


def clear_callbacks():
    for pv in [temperature_pv, colour_pv, wave_pv]:
        pv.clear_callbacks()
        pv.connection_callback = []

# Why PyEPICS?

## Because Python is awesome!

![https://xkcd.com/353/](images/xkcd-800.png)

* Easy to learn
* Highly readable syntax
* Powerful features
* Widely used at many research facilities

### PyEPICS enables you to...

* Quickly access data from EPICS PVs in convenient formats (eg numpy arrays)
* Easily create Python representations of physical devices
* Automate interactions with multiple devices
* Avoid the low-level details of the channel access protocol and focus on your application

![PyEPICS](images/pyepics.png)

# Setup

### Dependencies

* [EPICS Base](http://www.aps.anl.gov/epics/base/R3-14/12.php) libraries
* Python 2.6+ or 3.2+... ***2.7 or 3.4+ highly recommended***
* *Optional:* numpy for nicer array handling

### Installation

In your terminal:

```bash
pip3 install pyepics
```

Set the environment variables (eg in your `~/.bashrc` file):

```bash
export EPICS_BASE=/epics/base

export EPICS_HOST_ARCH=linux-x86_64
```

# Let's get some PVs

In [ ]:
import epics

In [ ]:
temperature_pv = epics.PV('PYEPICS_DEMO:TEMPERATURE')

In [ ]:
temperature_pv.get()

In [ ]:
temperature_pv.value

## Other PV properties

In [ ]:
temperature_pv.units

In [ ]:
print('Low alarm limit:', temperature_pv.lower_alarm_limit)
print('High alarm limit:', temperature_pv.upper_alarm_limit)

In [ ]:
print('Alarm severity:', temperature_pv.severity)

# Getting Stringy PVs

* Strings
* Enums
* Character arrays

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')
colour_pv.get()

In [ ]:
colour_pv.get(as_string=True)

In [ ]:
colour_pv.char_value

***Warning:*** `pv.char_value` has a bug where it only works after doing `pv.get(as_string=True)`

# Arrays and waveforms

In [ ]:
wave_pv = epics.PV('PYEPICS_DEMO:WAVE')
data = wave_pv.get()
data

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data)

# &nbsp;

# &nbsp;

# &nbsp;

## What happens if a PV doesn't exist

In [ ]:
invalid_pv = epics.PV('INVALID_NAME')

print(invalid_pv.get())

## Lesson

Don't do this:

```python
if not binary_pv.value:
    
    do_something()
```

Explicity check the value:

```python
if binary_pv.value == 0:
    
    do_something()
```

# Setting Values

In [ ]:
setpoint_pv = epics.PV('PYEPICS_DEMO:SETPOINT')

In [ ]:
setpoint_pv.put(5.2)

In [ ]:
setpoint_pv.value = 2.3

### Enum PVs accept int or str

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')

In [ ]:
colour_pv.put(1)

In [ ]:
colour_pv.put('red')

## Put doesn't guarantee the value was set

### When outside of drive limits

In [ ]:
setpoint_pv.put(15)  # PYEPICS_DEMO:SETPOINT has a DRVH of 10

In [ ]:
setpoint_pv.value

### When the PV is disconnected

In [ ]:
disconnected_pv = epics.PV('INVALID_NAME')

In [ ]:
status = disconnected_pv.put(3)
print(status)

# Callback functions

Notifications when PVs state changes:

* Connection
* Value
* Alarm

## Connection callbacks

In [ ]:
def on_connection_change(pvname, conn, pv):
    print(pvname, 'connected:', conn, flush=True)

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR',
                     connection_callback=on_connection_change)

## Value callbacks

In [ ]:
def on_value(pvname, value, char_value, timestamp, **kwargs):
    print(pvname, timestamp, value, char_value, flush=True)
    print('severity:', kwargs['severity'],
          'status:', kwargs['status'], flush=True)

#### Add the callback to the PV

In [ ]:
callback_id = colour_pv.add_callback(on_value)

#### Remove the callback

In [ ]:
colour_pv.remove_callback(callback_id)

#### Remove all callbacks

In [ ]:
colour_pv.clear_callbacks()

### What *not* to do inside callback functions

Any other channel access function calls

Slow / resource intensive processing; the callback should complete in ms

# Example of bad design

In [ ]:
from ipywidgets import Text

temperature_textbox = Text()
task_textbox = Text()

display(temperature_textbox, task_textbox)

def temperature_callback(value, **kwargs):
    temperature_textbox.value = str(value)
    
def colour_callback(char_value, **kwargs):
    task_textbox.value = 'RUNNING SLOW TASK'
    sleep(5)
    task_textbox.value = 'DONE!'

temperature_pv.add_callback(temperature_callback)
colour_pv.add_callback(colour_callback)

In [ ]:
temperature_pv.clear_callbacks()
colour_pv.clear_callbacks()

# Options

* Spawn a new thread from the callback

* Trigger processing on a worker thread using a queue or event 

## Deferring to a worker thread

In [ ]:
from threading import Event
COLOUR_CHANGE = Event()

display(temperature_textbox, task_textbox, )

def temperature_callback(value, **kwargs):
    temperature_textbox.value = str(value)
    
def colour_callback(char_value, **kwargs):
    COLOUR_CHANGE.set()
    
def worker():
    while True:
        COLOUR_CHANGE.wait()  # <= Blocks until event triggered
        task_textbox.value = 'RUNNING SLOW TASK'
        sleep(5)
        task_textbox.value = 'DONE!'
        COLOUR_CHANGE.clear()

worker_thread = epics.ca.CAThread(target=worker)
worker_thread.start()
        
temperature_pv.add_callback(temperature_callback)
colour_pv.add_callback(colour_callback)

In [ ]:
temperature_pv.clear_callbacks()
colour_pv.clear_callbacks()

## Devices

In [ ]:
demo_device = epics.Device(prefix='PYEPICS_DEMO:')

In [ ]:
demo_device.TEMPERATURE

In [ ]:
demo_device.COLOUR

In [ ]:
demo_device.get('COLOUR', as_string=True)

In [ ]:
demo_device.SETPOINT = 8

In [ ]:
demo_device.INVALID_NAME

## Defensive programming with the Device class

In [ ]:
demo_device = epics.Device(prefix='PYEPICS_DEMO:',
                           aliases={'target': 'SETPOINT',
                                    'value': 'READBACK'},
                           mutable=False)

In [ ]:
demo_device.target = 3.5

In [ ]:
demo_device.invalid_name = 9

## Subclassing Device

In [ ]:
class DemoDevice(epics.Device):
    def __init__(self, aliases=None, mutable=False, **kwargs):
        if aliases is None:
            aliases = {}
        aliases.update({
            'colour': 'COLOUR',
            'target': 'SETPOINT',
            'readback': 'READBACK'
        })
        super().__init__(aliases=aliases, mutable=mutable,
                         **kwargs)
    
    @property
    def colour_str(self):
        return self.get('colour', as_string=True)
    
    def go_to_setpoint(self, setpoint):
        self.target = setpoint
        while abs(self.readback - setpoint) > .1:
            print('Moving...', flush=True)
            sleep(.5)

In [ ]:
demo_device = DemoDevice(prefix='PYEPICS_DEMO:')

In [ ]:
demo_device.go_to_setpoint(.1)

In [ ]:
demo_device.colour_str

## Motor Device

```python
>>> from epics import Motor
>>> motor_x = Motor('SR00ID00USR00:MOT8')
>>> motor_x.readback
10.5
>>> motor_x.slew_speed = 100
>>> motor_x.move(123, wait=True)
0
```